In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext cython
%reload_ext cython
%load_ext autoreload

In [2]:
!snakemake
%autoreload
from editsim import *

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	all
	1

rule all:
    input: setup.py
    jobid: 0

/Users/kei/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'extra_compile_args'
  warnings.warn(msg)
/Users/kei/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'extra_link_args'
  warnings.warn(msg)
running build_ext
Finished job 0.
1 of 1 steps (100%) done
Complete log: /Users/kei/Projects/spykesim/drafts/.snakemake/log/2018-06-26T234139.138875.snakemake.log


In [3]:
nrow = 100
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)

# Simple editsim

## Simple

In [4]:
def pysimpleeditsim(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            dp_table[col1 + 1, col2 + 1] = np.max([
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match])
    return dp_table[-1, -1]

In [5]:
%timeit pysimpleeditsim(mat1, mat2)

67.2 ms ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit csimpleeditsim(mat1, mat2)

893 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
pysimpleeditsim(mat1, mat2) == csimpleeditsim(mat1, mat2)

True

## With bp

In [9]:
def pysimpleeditsim_withbp(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)            
    return dp_table[-1, -1], bp_table

In [10]:
%timeit pysimpleeditsim_withbp(mat1, mat2)

124 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit csimpleeditsim_withbp(mat1, mat2)

1.05 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
pysimpleeditsim_withbp(mat1, mat2)[0], csimpleeditsim_withbp(mat1, mat2)[0]

(205628.0, 205628.0)

In [16]:
(np.sum(pysimpleeditsim_withbp(mat1, mat2)[1] == csimpleeditsim_withbp(mat1, mat2)[1])) == (101*101)

True

## align

# local_exp editsim